In [1]:
import argparse
import pprint
import sys
import os
import json
import spotipy
import spotipy.util as util
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

client_id= "a82ef03cc3f44042b0f07c3c9220f175"
client_secret= "e049de0fb2a94fa9990ca6565ac7a053"
redirect_uri='http://localhost:8888/callback'
username='22am6rooejanltf555ip2dhxi'
playlist = '37i9dQZEVXbMXbN3EUUhlg' 
scope = 'user-library-read playlist-read-private'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

try:
    token = util.prompt_for_user_token(username, scope,client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
    sp=spotipy.Spotify(auth=token)
    print("Token Autorizada para " + username)
except:
    print('Token não accessível para ' + username)

    
# Obtém os dados da playlist em questão, oferecida pelo código da playlist

def get_playlist_content(username, playlist_id, sp):
    offset = 0
    songs = []
    while True:
        content = sp.user_playlist_tracks(username, playlist_id, fields=None,limit=100, offset=offset, market=None)
        songs += content['items']
        if content['next'] is not None:
            offset += 100
        else:
            break
    with open('{}-{}'.format(username, playlist_id), 'w') as outfile:
        json.dump(songs, outfile)

# Obtém os audio_features das músicas presentes na playlist, as features a serem obtidas são definidas na linha
# for features in audio_features... do método abaixo
# O mesmo método também exporta os dados obtidos para um planilha
        
def get_playlist_audio_features(username, playlist_id, sp):
    offset = 0
    songs = []
    items = []
    ids = []
    while True:
        content = sp.user_playlist_tracks(username, playlist_id, fields=None, limit=100, offset=offset, market=None)
        songs += content['items']
        if content['next'] is not None:
            offset += 100
        else:
            break
    for i in songs:
        ids.append(i['track']['id'])
        index = 0
        audio_features = []
        while index < len(ids):
            audio_features += sp.audio_features(ids[index:index + 50])
            index += 50
            features_list = []
            for features in audio_features:
                features_list.append([features['energy'], features['liveness'],
                features['tempo'], features['speechiness'],
                features['acousticness'], features['instrumentalness'],
                features['time_signature'], features['danceability'],
                features['key'], features['duration_ms'],
                features['loudness'], features['valence'],
                features['mode'], features['type'],
                features['uri']])
                df = pd.DataFrame(features_list, columns=['energy', 'liveness',
                                                          'tempo', 'speechiness',
                                                          'acousticness', 'instrumentalness',
                                                          'time_signature', 'danceability',
                                                          'key', 'duration_ms', 'loudness',
                                                          'valence', 'mode', 'type', 'uri'])
                df.to_csv('{}-{}.csv'.format(username, playlist_id), index=False)
        
        
# Chama os métodos de obtenção do conteúdo e das audio_features de uma playlist
def main(username, playlist):
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    print ("Adquirindo conteúdo da playlist")
    get_playlist_content(username, playlist, sp)
    print ("Adquirindo recursos de aúdio da playlist")
    get_playlist_audio_features(username, playlist, sp)
    print ("Playlist salva como " +username + "-" + playlist + " no formato [Arquivo de Valores Separados por Vírgulas do Microsoft Excel]")

main(username, playlist)

Token Autorizada para 22am6rooejanltf555ip2dhxi
Adquirindo conteúdo da playlist
Adquirindo recursos de aúdio da playlist
Playlist salva como 22am6rooejanltf555ip2dhxi-37i9dQZEVXbMXbN3EUUhlg no formato [Arquivo de Valores Separados por Vírgulas do Microsoft Excel]
